In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\Chcfpp01\Groups\HTS\Code_Repository\Python\Libraries'
Site_Packages = r'c:\users\mr068144\appdata\roaming\python\python36\site-packages'
import sys, os, arcpy, csv, json, math
print(sys.executable)
print(sys.version)
for p in [HSMPY_PATH,Site_Packages]:
    if not p in sys.path:
        sys.path.append(p)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime
from datetime import timedelta
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import zipfile
import shutil
import scipy
from ipywidgets import IntProgress
from time import gmtime, strftime
#from pysal.esda.mapclassify import Natural_Breaks as nb
import matplotlib.patches as patches
print(strftime("%Y-%m-%d %H:%M:%S"))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
2019-04-26 09:40:19


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'
PythonDir   = Project_DIR + '\\6.Python_Notebook'
DeliverableDir = Project_DIR + '\\8.Deliverable'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

Src_Ped_Loc  = ReceivedDir + '\\Ped_2017_KAB\\crashkabped.csv'
Src_Ped_Unit = ReceivedDir + '\\Ped_2017_KAB\\vehiclekabped.csv'
Src_Ped_Occ  = ReceivedDir + '\\Ped_2017_KAB\\occkabped.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

FOU_2017_105 = ReceivedDir + '\\v105\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'
FOU_2017_10  = ReceivedDir + '\\v10\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'

FHWA_HPMS     = {year:ReceivedDir + '\\FHWA_HPMS\\newmexico{}\\NewMexico{}.shp'.format(year,year) for year in Years}
MS2_HPMS_2017 = {year:ReceivedDir + '\\NM_HPMS_N2017_A{}.csv'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}

StreetNames_SHP = ReceivedDir + "\\streetNames\AltStreetName.shp"

UTM13N   = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

2019-04-26 09:40:19


In [3]:
# Reading Source Crashes
print(strftime("%Y-%m-%d %H:%M:%S"))

Crash_DF = hsmpy3.common.FCtoDF(Project_GDB + '\\NM_Crash_KAB_12_16',readGeometry=True)
Crash_DF.index = Crash_DF.CID

Loc_DF = pd.read_csv(Src_Loc,low_memory=False)
Loc_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Loc_DF.year,Loc_DF.UCRnumber)]
Loc_DF.index = Loc_DF.CID
Loc_DF = Loc_DF.sort_index()

F = ['CrashDirection',
 'Astreet',
 'Bstreet',
 'Landmark',
 'DirectionFromLandmark',
 'Measurement',
 'MeasurementUnit',
 'Class',
 'Analysis',
 'CrashOccurrence',
 'HitRun',
 'Light',
 'Weather',
 'RoadCharacter',
 'RoadGrade',
 'nVeh',
 'TopCFacc',
 'Alcinv',
 'Druginv',
 'MCinv',
 'PECinv',
 'TRKinv',
 'HZinv',
 'TribalJurisdiction']
Loc_DF.index = Loc_DF.CID
for c in F:
    Crash_DF[c] = Loc_DF[c].loc[Crash_DF.index]
Crash_DF['OnCurve'] = Crash_DF.RoadCharacter.apply(lambda x:1 if x==2 else 0)
print(Crash_DF.shape)

Unit_DF = pd.read_csv(Src_Unit,low_memory=False)
Unit_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)]
Unit_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Unit_DF.year, Unit_DF.UCRnumber)]
#Unit_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)],names=['CID','Unit'])
Unit_DF.index = Unit_DF.CID
Unit_DF = Unit_DF.sort_index()
Unit_DF = Unit_DF[Unit_DF.CID.isin(Crash_DF.CID)]
Unit_DF['Shape'] = Crash_DF.Shape.loc[Unit_DF.index]
Unit_DF['Date'] = Crash_DF.Date.loc[Unit_DF.index]
F = ['VehNo','Date', 'Passengers', 'DALC', 'DRUG', 'DRESID', 'TopCFcar', 'TypeV', 'Belt', 'Helmet', 'VehDirection', 'StreetOn', 'PostedSpeed', 'SafeSpeed', 'LeftScene', 'DrSeatPos', 'DrAge', 'DrSex', 'DrInjuryCode', 'DrOPCode', 'DrOPProperlyUsed', 'DrAirbagDeployed', 'DrEjected', 'DrEMSNum', 'DrMedTrans', 'DLState', 'DLType', 'DLRestrictions', 'DLExpires', 'DLEndorsements', 'DLStatus', 'VeYear', 'VeMake', 'VeColor', 'VeBodystyle', 'VeCargoBody', 'VeUse1', 'VeUse2', 'VeLicPlateRegYr', 'VeLicPlateState', 'VeTowed', 'VeTowedDisabled', 'VeDamageSeverity', 'VeDamageExtent', 'VeDamageAll', 'HazmatName', 'HazmatPlacard', 'HazmatReleased', 'RoadConditionsVe', 'RoadSurfaceVe', 'TrafficControlDevice', 'RoadDesignLanes', 'RoadDesignDivider', 'RoadDesign', 'SequenceEvent1', 'SequenceEvent2', 'SequenceEvent3', 'SequenceEvent4', 'vVehNo', 'vViolation', 'vAction', 'TraCS', 'CUID', 'CID', 'Shape']
Unit_DF = Unit_DF[F]
print(Unit_DF.shape)


OL = ['CUID','PPLNo', 'OccNo', 'SeatPos', 'Age', 'Sex', 'Injury', 'OPCode', 'OPProperlyUsed', 'AirbagDeployed', 'Ejected', 'MedTrans', 'DAparked', 'Belt', 'Helmet']
Occ_DF = pd.read_csv(Src_Occ,low_memory=False)
Occ_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Occ_DF.year, Occ_DF.UCRnumber)]
Occ_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)]
#Occ_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v,p) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)],names=['CID','Unit','Person'])
Occ_DF.index = Occ_DF.CID
Occ_DF = Occ_DF[Occ_DF.CID.isin(Crash_DF.CID)]
Occ_DF = Occ_DF.sort_index()
Occ_DF['Shape'] = Crash_DF.Shape.loc[Occ_DF.index]
Occ_DF['Date'] = Crash_DF.Date.loc[Occ_DF.index]
F = ['VehNo','Date', 'PPLNo', 'OccNo', 'SeatPos', 'Age', 'Sex', 'Injury', 'OPCode', 'OPProperlyUsed', 'AirbagDeployed', 'Ejected', 'MedTrans', 'DAparked', 'Belt', 'Helmet', 'CID', 'CUID', 'Shape']
Occ_DF = Occ_DF[F]
print(Occ_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-26 09:40:19
(22903, 51)
(40141, 65)
(59513, 19)
2019-04-26 09:43:18


In [4]:
# Translate and Add Additional Fields to Crash Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Crash_DF['X'] = [s.firstPoint.X for s in Crash_DF.Shape]
Crash_DF['Y'] = [s.firstPoint.Y for s in Crash_DF.Shape]
idx = Crash_DF[(~pd.isnull(Crash_DF.X)) & (~pd.isnull(Crash_DF.Y))].index
Crash_DF.loc[idx,'Point'] = [arcpy.PointGeometry(arcpy.Point(x,y),hsmpy3.nm.NAD83_NM).projectAs(hsmpy3.common.WGS1984) for x,y in zip(list(Crash_DF.loc[idx,'X']),list(Crash_DF.loc[idx,'Y']))]
def GetLON(p):
        try:
            return(p.X)
        except:
            pass
def GetLAT(p):
        try:
            return(p.Y)
        except:
            pass
Crash_DF.loc[idx,'LAT'] = [GetLAT(p) for p in Crash_DF.Point.loc[idx]]
Crash_DF.loc[idx,'LON'] = [GetLON(p) for p in Crash_DF.Point.loc[idx]]
    
Crash_DF['SUN_ANG'] = [hsmpy3.il.SunAngle(lt,ln,dt) for lt,ln,dt in zip(list(Crash_DF.LAT),list(Crash_DF.LON),list(Crash_DF.Date))]

def DayNight(d,s):
    if s<-6:
        return('Night')
    if s>0:
        return('Day')
    if s>=-6 and s <=0:
        if d.hour<12:
            return('Dawn')
        else:
            return('Dusk')
    return('Unknown')
Crash_DF["DayNight"] = [DayNight(d,s) for d,s in zip(Crash_DF.Date,Crash_DF.SUN_ANG)]

Crash_DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in Crash_DF.Date]
TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
Crash_DF['DayName'] = [d.weekday_name for d in Crash_DF.Date]
DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Crash_DF['Month']  = [d.strftime('%b') for d in Crash_DF.Date]
MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

Crash_DF['Year'] = [d.year for d in Crash_DF.Date]
Crash_DF['PedCrashCount'] = [{True:1,False:0}[sum([k,a,b])>0] for k,a,b in zip(Crash_DF.PedK,Crash_DF.PedA,Crash_DF.PedB)]
Crash_DF['Count'] = 1
Crash_DF['KAB'] = Crash_DF.Killed + Crash_DF.ClassA + Crash_DF.ClassB
def LightConv(l):
    if pd.isnull(l):
        return('Unknown')
    else:
        return({1:'DayLight',2:'Dawn',3:'Dusk',4:'Dark - Lighted',5:'Dark - unlighted',6:'Unknown',0:'Unknown',99:'Unknown',98:'Unknown'}[l])
def WeatherConv(l):
    if pd.isnull(l):
        return('Unknown')
    else:
        return({1:'Clear',2:'Raining',3:'Snowing',4:'Fog',5:'Dust',6:'Wind',7:'Other',8:'Sleet or Hail',99:'Unknown',98:'Unknown',0:'Unknown'}[l])
    
Crash_DF['Light'] = [LightConv(l) for l in Loc_DF.Light.loc[Crash_DF.index]]
Crash_DF['Weather'] = [WeatherConv(l) for l in Loc_DF.Weather.loc[Crash_DF.index]]
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-26 09:43:18
2019-04-26 09:54:57


In [5]:
# Reading the Final Network Screening Results
print(strftime("%Y-%m-%d %H:%M:%S"))
SHP_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\8.Deliverable\Shapefiles'
Ped_FC = SHP_Dir + '\\State_Pedestrian.shp' 
Seg_FC = SHP_Dir + '\\State_Segment.shp' 
Int_FC = SHP_Dir + '\\State_Int.shp'
IntD_DF = hsmpy3.common.FCtoDF(Int_FC,readGeometry=True)
SegD_DF = hsmpy3.common.FCtoDF(Seg_FC,readGeometry=True)
PedD_DF = hsmpy3.common.FCtoDF(Ped_FC,readGeometry=True)
SegD_DF['Mileage'] = SegD_DF.Shape.apply(lambda x:x.length/5280.0)
def TranslateUrban(x):
    codes = {0:'Rural',
            1171:'Albuquerque',
            27253:'El Paso (Partial)',
            29278:'Farmington',
            47935:'Las Cruces',
            51499:'Los Lunes',
            79363:'Santa Fe',
            99998:'Small Urban',
            99999:'Rural'}
    return(codes[x])
SegD_DF['UrbanRural'] = SegD_DF.URBAN_CODE.apply(TranslateUrban)
PedD_DF['UrbanRural'] = PedD_DF.URBAN_CODE.apply(TranslateUrban)
IntD_DF['UrbanRural'] = IntD_DF.I_URBANCOD.apply(TranslateUrban)

SegD_DF['StateLocal'] = SegD_DF.OWNERSHIP.apply(lambda x:'State' if x==1 else 'Other')
PedD_DF['StateLocal'] = PedD_DF.OWNERSHIP.apply(lambda x:'State' if x==1 else 'Other')
IntD_DF['StateLocal'] = IntD_DF.apply(lambda row:'State' if (row.OWNERSHIP1==1 or row.OWNERSHIP_==1) else 'Other',axis=1)


SegD_DF['Dis_Label'] = SegD_DF.apply(lambda row:'{} ({})'.format(row.RID.replace('-',''),row.RoadLabel),axis=1)
PedD_DF['Dis_Label'] = PedD_DF.apply(lambda row:'{} ({})'.format(row.RID.replace('-',''),row.RoadLabel),axis=1)

PedD_DF['Mileage'] = PedD_DF.Shape.apply(lambda x:x.length/5280.0)
IntD_DF.index = IntD_DF.Int_ID
IntD_DF = IntD_DF.sort_values('CF_Obs',ascending=False)
SegD_DF = SegD_DF.reset_index()
idx = pd.IntervalIndex.from_tuples([(bmp,emp) for bmp,emp in zip(SegD_DF.BMP,SegD_DF.EMP)],'left')
SegD_DF.index = pd.MultiIndex.from_arrays([SegD_DF.RID.apply(lambda x:x.replace('-','')),idx],names = ['Route ID','Milepost'])
SegD_DF = SegD_DF.sort_index()
IntD_DF = IntD_DF[IntD_DF.District.apply(len)>1]
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-26 09:54:57
2019-04-26 09:55:41


In [6]:
# Create Results GDB
ResultsGDB = DeliverableDir + '\\GDB3.gdb'
# Create GDB
print('[{}] create file gdb'.format(strftime("%Y-%m-%d %H:%M:%S")))
if not os.path.exists(ResultsGDB):
    arcpy.CreateFileGDB_management(out_folder_path=os.path.dirname(ResultsGDB),out_name=os.path.basename(ResultsGDB))
    print('[{}] done: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),ResultsGDB))
else:
    print('[{}] already exists: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),ResultsGDB))

[2019-04-26 09:55:41] create file gdb
[2019-04-26 09:55:41] already exists: \\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\8.Deliverable\GDB3.gdb


In [22]:
# Adding Seg, Int, Ped, and Crashes to GDB
Temp_FC = Seg_FC
Temp_FC_Name = 'Segments'
try:
    print('[{}] deleting current table: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))
    arcpy.management.Delete(os.path.join(ResultsGDB ,Temp_FC_Name))
except:
    pass
print('[{}] add {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Temp_FC_Name))
arcpy.conversion.FeatureClassToFeatureClass(in_features=Temp_FC,out_path=ResultsGDB,out_name=Temp_FC_Name)
print('[{}] done: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))

Temp_FC = Int_FC
Temp_FC_Name = 'Intersections'
try:
    print('[{}] deleting current table: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))
    arcpy.management.Delete(os.path.join(ResultsGDB ,Temp_FC_Name))
except:
    pass
print('[{}] add {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Temp_FC_Name))
arcpy.conversion.FeatureClassToFeatureClass(in_features=Temp_FC,out_path=ResultsGDB,out_name=Temp_FC_Name)
print('[{}] done: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))

Temp_FC = Ped_FC
Temp_FC_Name = 'Pedestrian'
try:
    print('[{}] deleting current table: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))
    arcpy.management.Delete(os.path.join(ResultsGDB ,Temp_FC_Name))
except:
    pass
print('[{}] add {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Temp_FC_Name))
arcpy.conversion.FeatureClassToFeatureClass(in_features=Temp_FC,out_path=ResultsGDB,out_name=Temp_FC_Name)
print('[{}] done: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))

#Temp_FC = Project_GDB + '\\NM_Crash_KAB_12_16'
#Temp_FC_Name = 'NM_Crash_KAB_12_16'
#try:
#    print('[{}] deleting current table: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))
#    arcpy.management.Delete(os.path.join(ResultsGDB ,Temp_FC_Name))
#except:
#    pass
#print('[{}] add {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Temp_FC_Name))
#arcpy.conversion.FeatureClassToFeatureClass(in_features=Temp_FC,out_path=ResultsGDB,out_name=Temp_FC_Name)
#print('[{}] done: {}'.format(strftime("%Y-%m-%d %H:%M:%S"),arcpy.GetCount_management(os.path.join(ResultsGDB ,Temp_FC_Name))))

hsmpy3.common.ConvertPandasDFtoTable(DF=Crash_DF,Table=ResultsGDB + '\\NM_Crash_KAB_12_16',shape='Shape')
hsmpy3.common.ConvertPandasDFtoTable(DF=Unit_DF,Table=ResultsGDB + '\\NM_Vehicle_KAB_12_16',shape='Shape')
hsmpy3.common.ConvertPandasDFtoTable(DF=Occ_DF,Table=ResultsGDB + '\\NM_Occupant_KAB_12_16',shape='Shape')

[2019-04-15 14:39:08] deleting current table: 2991
[2019-04-15 14:39:10] add Segments
[2019-04-15 14:39:14] done: 2991
[2019-04-15 14:39:15] deleting current table: 7184
[2019-04-15 14:39:17] add Intersections
[2019-04-15 14:39:22] done: 7184
[2019-04-15 14:39:23] deleting current table: 1539
[2019-04-15 14:39:26] add Pedestrian
[2019-04-15 14:39:29] done: 1539
[2019-04-15 14:39:30] create table
[2019-04-15 14:39:30] deleting current table: 0
[2019-04-15 14:39:33] spatial reference detected: NAD_1983_2011_StatePlane_New_Mexico_Central_FIPS_3002_Ft_US
[2019-04-15 14:39:33] geometry detected: POINT
[2019-04-15 14:39:37] adding fields
[2019-04-15 14:43:03] inserting rows
[2019-04-15 14:45:25] done! 22903
[2019-04-15 14:45:26] create table
[2019-04-15 14:45:27] spatial reference detected: NAD_1983_2011_StatePlane_New_Mexico_Central_FIPS_3002_Ft_US
[2019-04-15 14:45:28] geometry detected: POINT
[2019-04-15 14:45:31] adding fields
[2019-04-15 14:48:30] inserting rows
[2019-04-15 14:52:14] do

In [7]:
# Create Crash DataFrames
Crash_DF.index = Crash_DF.RID_P
SegD_DF.index = SegD_DF.SiteID
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP) & (df.Int_Related==0)]
    return(df.CID.tolist())
SegD_DF['CIDs'] = SegD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.SiteID,cid) for i,r in SegD_DF.iterrows() for cid in r.CIDs],columns=['SiteID','CID'])
df.index = df.SiteID
SegD_Crash_DF = df.sort_index()

PedD_DF.index = PedD_DF.SiteID
Crash_DF['PedInv'] = Crash_DF.apply(lambda row:sum([row.PedK,row.PedA,row.PedB])>0,axis=1)
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP) & (df.PedInv)]
    return(df.CID.tolist())
PedD_DF['CIDs'] = PedD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.SiteID,cid) for i,r in PedD_DF.iterrows() for cid in r.CIDs],columns=['SiteID','CID'])
df.index = df.SiteID
PedD_Crash_DF = df.sort_index()

IntD_DF.index = IntD_DF.Int_ID
def GetCrashCIDs(row):
    df = Crash_DF[(Crash_DF.Int_ID==row.Int_ID)]
    return(df.CID.tolist())
IntD_DF['CIDs'] = IntD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.Int_ID,cid) for i,r in IntD_DF.iterrows() for cid in r.CIDs],columns=['Int_ID','CID'])
df.index = df.Int_ID
IntD_Crash_DF = df.sort_index()
Crash_DF.index = Crash_DF.CID

In [24]:
# Create Crash Relationships
R_Seg_Crash = ResultsGDB + '\\R_Seg_Crash'
arcpy.Delete_management(R_Seg_Crash)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\Segments',
    destination_table      = ResultsGDB + '\\NM_Crash_KAB_12_16', 
    out_relationship_class = R_Seg_Crash, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Crash", 
    backward_label         = "Segments", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "SiteID", 
    origin_foreign_key     = "SiteID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)
uc = arcpy.InsertCursor(R_Seg_Crash)
for i,r in SegD_Crash_DF.iterrows():
    row = uc.newRow()
    row.setValue('SiteID',r.SiteID)
    row.setValue('CID',r.CID)
    uc.insertRow(row)
del uc
print('[{}] Seg - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

R_Ped_Crash = ResultsGDB + '\\R_Ped_Crash'
arcpy.Delete_management(R_Ped_Crash)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\Pedestrian',
    destination_table      = ResultsGDB + '\\NM_Crash_KAB_12_16', 
    out_relationship_class = R_Ped_Crash, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Crash", 
    backward_label         = "Pedestrian", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "SiteID", 
    origin_foreign_key     = "SiteID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)
uc = arcpy.InsertCursor(R_Ped_Crash)
for i,r in PedD_Crash_DF.iterrows():
    row = uc.newRow()
    row.setValue('SiteID',r.SiteID)
    row.setValue('CID',r.CID)
    uc.insertRow(row)
del uc
print('[{}] Ped - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

R_Int_Crash = ResultsGDB + '\\R_Int_Crash'
arcpy.Delete_management(R_Int_Crash)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\Intersections',
    destination_table      = ResultsGDB + '\\NM_Crash_KAB_12_16', 
    out_relationship_class = R_Int_Crash, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Crash", 
    backward_label         = "Intersections", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "Int_ID", 
    origin_foreign_key     = "Int_ID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)
uc = arcpy.InsertCursor(R_Int_Crash)
for i,r in IntD_Crash_DF.iterrows():
    row = uc.newRow()
    row.setValue('Int_ID',r.Int_ID)
    row.setValue('CID',r.CID)
    uc.insertRow(row)
del uc
print('[{}] Int - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

R_Crash_Unit = ResultsGDB + '\\R_Crash_Unit'
arcpy.Delete_management(R_Crash_Unit)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\NM_Crash_KAB_12_16',
    destination_table      = ResultsGDB + '\\NM_Vehicle_KAB_12_16', 
    out_relationship_class = R_Crash_Unit, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Vehicle", 
    backward_label         = "Crash", 
    message_direction      = "NONE", 
    cardinality            = "ONE_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "CID", 
    origin_foreign_key     = "CID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)
print('[{}] Crash - Vehicle done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

R_Unit_Occ = ResultsGDB + '\\R_Unit_Occ'
arcpy.Delete_management(R_Unit_Occ)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\NM_Vehicle_KAB_12_16',
    destination_table      = ResultsGDB + '\\NM_Occupant_KAB_12_16', 
    out_relationship_class = R_Unit_Occ, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Person", 
    backward_label         = "Vehicle", 
    message_direction      = "NONE", 
    cardinality            = "ONE_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "CUID", 
    origin_foreign_key     = "CUID", 
    destination_primary_key= "CUID", 
    destination_foreign_key= 'CUID'
)
print('[{}] Vehicle - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

[2019-04-15 14:55:42] Seg - Crash done!
[2019-04-15 14:55:47] Ped - Crash done!
[2019-04-15 14:55:57] Int - Crash done!
[2019-04-15 14:56:02] Crash - Vehicle done!
[2019-04-15 14:56:08] Vehicle - Crash done!


In [8]:
# Read Segments - used for Int App Shapes
print('[{}] read segments'.format(strftime("%Y-%m-%d %H:%M:%S")))
Seg_2016 = hsmpy3.common.FCtoDF(Segs[2016])
Seg_2016.index = Seg_2016.ROUTE_ID.apply(lambda x:x.replace('-',''))
print('[{}] - {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Seg_2016.shape))

[2019-04-26 09:56:09] read segments
[2019-04-26 10:02:55] - (224751, 17)


In [9]:
# Create Int App Data Frame
def extractLabel(x):
    try:
        M = x.split(';')
        N = []
        for s in M:
            L = s.split('@')
            a = L[1].find('(')+1
            b = L[1][a:-1]
            c = float(L[1][:a-1])
            a = L[0]
            N.append(['{} ({})'.format(a,b),c,a])
        return(N)
    except:
        return([])
        print(x)
def QuerySeg1(row):
    df = Seg_2016.loc[row.RID]
    df = df[(df.BEG_POINT<=row.MP) & (df.END_POINT>=row.MP)]
    try:
        return(list(df.OWNERSHIP)[0])
    except:
        pass
def QuerySeg2(row):
    df = Seg_2016.loc[row.RID]
    df = df[(df.BEG_POINT<=row.MP) & (df.END_POINT>=row.MP)]
    try:
        return(list(df.URBAN_CODE)[0])
    except:
        pass

def TranslateUrban(x):
    codes = {0:'Rural',
            1171:'Albuquerque',
            27253:'El Paso (Partial)',
            29278:'Farmington',
            47935:'Las Cruces',
            51499:'Los Lunes',
            79363:'Santa Fe',
            99998:'Small Urban',
            99999:'Rural'}
    return(codes[x])

print('[{}] Create Intersection Approach Dataframe'.format(strftime("%Y-%m-%d %H:%M:%S")))
df = IntD_DF
sdf = pd.DataFrame(df.ApprLabels.apply(extractLabel))
sdf['RIDList'] = IntD_DF.RIDList.apply(lambda x:';'.join(list(set([i for i in x.split(';') if i[-1]=='P'])))).loc[sdf.index]
sdf = sdf.reset_index()
cdf = pd.DataFrame(index = pd.Index([b[0] for i in sdf.ApprLabels for b in i],name='Label'),
                   data={'Int_ID':[r.Int_ID for i,r in sdf.iterrows() for x in r.ApprLabels],
                         'RID'   :[b[2] for i in sdf.ApprLabels for b in i],
                         'MP'    :[b[1] for i in sdf.ApprLabels for b in i],
                        })

cdf['StateLocal'] = cdf.apply(QuerySeg1,axis=1).fillna(0).astype(int).apply(lambda x:'State' if x==1 else 'Other')
cdf['UrbanRural'] = IntD_DF.loc[cdf.Int_ID.tolist(),'UrbanRural'].tolist()
cdf['District'] = IntD_DF.loc[cdf.Int_ID.tolist(),'District'].tolist()
cdf['Shape'] = IntD_DF.loc[cdf.Int_ID.tolist(),'Shape'].tolist()
cdf.index.name = 'Dis_Label'
cdf['CIDs'] = IntD_DF.CIDs.loc[cdf.Int_ID.tolist()].tolist()
cdf['SafetyTier'] = IntD_DF.SafetyTier.loc[cdf.Int_ID.tolist()].tolist()
def Fun1(row):
    rid = row.RID
    int_id = row.Int_ID
    ridL = sdf.loc[int_id,'RIDList'].split(';')
    #return(ridL)
    for r in ridL:
        if r.replace('-','')==rid:
            return(r)
sdf.index = sdf.Int_ID
cdf['ROUTE_ID'] = cdf.apply(Fun1,axis=1)
cdf = cdf.reset_index()
IntApp_DF = cdf

df = IntApp_DF[['ROUTE_ID','MP','Int_ID']]
df['BMP'] = df.MP-250.0/5280.0
df['EMP'] = df.MP+250.0/5280.0
df['BMP'] = df.BMP.apply(lambda x:0 if x<0 else x)
df = df.rename(columns={'RID':'ROUTE_ID','BMP':"BEG_POINT",'EMP':'END_POINT'})
hsmpy3.common.ConvertPandasDFtoTable(DF=df,Table=Project_GDB+'\\Int_App_Table')
hsmpy3.network.CreateRouteEventLayer(AttTable=Project_GDB+'\\Int_App_Table',
                                     Sites_Routes=Routes[2016],
                                     BMP='BEG_POINT',EMP='END_POINT',RouteID='ROUTE_ID',
                                     Fields=['Int_ID'],Output=Project_GDB+'\\Int_App_Event')
hsmpy3.common.PrintSummary(Project_GDB+'\\Int_App_Event')
df = hsmpy3.common.FCtoDF(Project_GDB+'\\Int_App_Event',readGeometry=True)
s = df.groupby('Int_ID').Shape.aggregate(lambda L:arcpy.Polyline(arcpy.Array([l[0] for l in L])))
IntApp_DF.index = IntApp_DF.Int_ID
IntApp_DF['Shape'] = s.loc[IntApp_DF.index]
print('[{}] - {}'.format(strftime("%Y-%m-%d %H:%M:%S"),IntApp_DF.shape))

[2019-04-26 10:02:55] Create Intersection Approach Dataframe
[2019-04-26 10:04:02] create table


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

[2019-04-26 10:04:02] deleting current table: 11178
[2019-04-26 10:07:57] adding fields
[2019-04-26 10:10:55] inserting rows
[2019-04-26 10:11:21] done! 11178
Type: Polyline
Columns: 8 x Rows: 10458
['BEG_POINT', 'END_POINT', 'Int_ID', 'LOC_ERROR', 'OBJECTID', 'ROUTE_ID', 'Shape', 'Shape_Length']
[2019-04-26 10:18:23] - (11178, 11)


In [10]:
# Aggregate the locations by district, ownership, rural/urban, road name
def merge_dates(grp):
    grp = grp.sort_values('BMP')
    dt_groups = (grp['BMP'] != grp['EMP'].shift()).cumsum()
    df = grp.groupby(dt_groups).agg({'BMP': 'first', 'EMP': 'last'})
    return(';'.join(['{:.2f}:{:.2f}'.format(r.BMP,r.EMP) for i,r in df.iterrows()]))
def CrossInt(df):
    l = df.Dis_Label.unique().tolist()[0]
    L = IntApp_DF[(IntApp_DF.Dis_Label==l) & (IntApp_DF.SafetyTier.isin(Int_Sel_Tiers2))].Int_ID.tolist()
    def ListApp(idf):
        df1 = idf[idf.Dis_Label==l]
        #if df1.shape[0]==1:
        mp = df1.MP.tolist()[0]
        return([mp,';'.join([r.Dis_Label for i,r in idf[idf.Dis_Label!=l].iterrows()])])

    s = pd.DataFrame(IntApp_DF[(IntApp_DF.Int_ID.isin(L))].groupby('Int_ID').apply(ListApp))
    #display(l)
    s.columns = ['MP_Label']
    s = s.reset_index()
    s['MP'] = s.MP_Label.apply(lambda x:x[0])
    s['Label'] = s.MP_Label.apply(lambda x:x[1])
    s = s.sort_values('MP')
    #display(s)
    return(';'.join(['@{:.3f},ID:{},{}'.format(r.MP,r.Int_ID,r.Label) for i,r in s.iterrows()]))
def MergeCIDs(S):
    l = []
    for i in S:
        l.extend(i)
    return(l)

Seg_Sel_Tiers1 = [1]
Ped_Sel_Tiers1 = [1]
Int_Sel_Tiers1 = [1]

Seg_Sel_Tiers2 = [1,2]
Ped_Sel_Tiers2 = [1,2]
Int_Sel_Tiers2 = [1,2]

L1 = SegD_DF[SegD_DF.SafetyTier.isin(Seg_Sel_Tiers1)].Dis_Label.unique().tolist()
L2 = PedD_DF[PedD_DF.SafetyTier.isin(Ped_Sel_Tiers1)].Dis_Label.unique().tolist()
L3 = IntApp_DF[IntApp_DF.SafetyTier.isin(Int_Sel_Tiers1)].Dis_Label.unique().tolist()

L = list(set(L1 + L2 + L3))
Res_DF = pd.DataFrame()
Sel = SegD_DF[(SegD_DF.Dis_Label.isin(L)) & (SegD_DF.CIDs.apply(len)>0) & (SegD_DF.District.apply(len)>1)] 
Sel2 = PedD_DF[(PedD_DF.Dis_Label.isin(L)) & (PedD_DF.CIDs.apply(len)>0) & (PedD_DF.District.apply(len)>1)] 
Sel3 = IntApp_DF[(IntApp_DF.Dis_Label.isin(L)) & (IntApp_DF.CIDs.apply(len)>0) & (IntApp_DF.District.apply(len)>1)] 
idx1 = Sel[(Sel.SafetyTier.isin(Seg_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Mileage.sum().round(2).index
idx2 = Sel2[(Sel2.SafetyTier.isin(Ped_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Mileage.sum().round(2).index
idx3 = Sel3[(Sel3.SafetyTier.isin(Int_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).size().index
idx = idx1.append(idx2).append(idx3)
Res_DF = pd.DataFrame(index = idx.drop_duplicates())
Res_DF['PSI_MPs']     = Sel[(Sel.SafetyTier.isin(Seg_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).apply(merge_dates).loc[Res_DF.index]
Res_DF['PSI_Mileage'] = Sel[(Sel.SafetyTier.isin(Ped_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Mileage.sum().round(2).loc[Res_DF.index]
Res_DF['Ped_MPs']     = Sel2[(Sel2.SafetyTier.isin(Seg_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).apply(merge_dates).loc[Res_DF.index]
Res_DF['Ped_Mileage'] = Sel2[(Sel2.SafetyTier.isin(Ped_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Mileage.sum().round(2).loc[Res_DF.index]
Res_DF['Num_Int']     = Sel3[(Sel3.SafetyTier.isin(Int_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).apply(lambda df:len(set(df.Int_ID.tolist()))).loc[Res_DF.index].fillna(0).astype(int)
Res_DF['Int_MPs']     = Sel3[(Sel3.SafetyTier.isin(Int_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).apply(CrossInt).loc[Res_DF.index]

Res_DF['PSI_Shape'] = Sel[(Sel.SafetyTier.isin(Seg_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Shape.aggregate(lambda L:arcpy.Polyline(arcpy.Array([prt for line in L for prt in line]))).loc[Res_DF.index]
Res_DF['Ped_Shape'] = Sel2[(Sel2.SafetyTier.isin(Ped_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Shape.aggregate(lambda L:arcpy.Polyline(arcpy.Array([prt for line in L for prt in line]))).loc[Res_DF.index]
Res_DF['Int_Shape'] = Sel3[(Sel3.SafetyTier.isin(Int_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).Shape.aggregate(lambda L:arcpy.Polyline(arcpy.Array([prt for line in L for prt in line]))).loc[Res_DF.index]

Res_DF['Shape'] = ''
for i,row in Res_DF.iterrows():
    L = []
    if not pd.isnull(row.PSI_Shape):
        if row.PSI_Shape.length>0:
            L.append(row.PSI_Shape)
    if not pd.isnull(row.Ped_Shape):
        if row.Ped_Shape.length>0:
            L.append(row.Ped_Shape)
    if not pd.isnull(row.Int_Shape):
        if row.Int_Shape.length>0:
            L.append(row.Int_Shape)
    if len(L)>0:
        pl = arcpy.Polyline(arcpy.Array([prt for line in L for prt in line]))
        Res_DF.set_value(i,'Shape',pl)
Res_DF['Shape'] = Res_DF.Shape.apply(lambda p:p.union(p))
Res_DF['Total_Mileage'] = Res_DF.Shape.apply(lambda x:x.length/5280.0)


Res_DF['PSI_CIDs'] = Sel[(Sel.SafetyTier.isin(Seg_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).CIDs.apply(MergeCIDs)
Res_DF['Ped_CIDs'] = Sel2[(Sel2.SafetyTier.isin(Ped_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).CIDs.apply(MergeCIDs)
Res_DF['Int_CIDs'] = Sel3[(Sel3.SafetyTier.isin(Int_Sel_Tiers2))].groupby(['District','StateLocal','UrbanRural','Dis_Label']).CIDs.apply(MergeCIDs)
for c in ['Ped_CIDs','PSI_CIDs','Int_CIDs']:
    Res_DF.loc[Res_DF[c].isnull(),[c]] = Res_DF.loc[Res_DF[c].isnull(),c].apply(lambda x: [])
Res_DF['All_CIDs'] = ''
#Res_DF['All_CIDs'] = Res_DF['All_CIDs'].astype(object)
for i,r in Res_DF.iterrows():
    l = list(set(r.PSI_CIDs+r.Ped_CIDs+r.Int_CIDs))
    Res_DF.set_value(i,'All_CIDs',l)

for sev in ['K','A','B']:
    Crash_DF['Seg_{}_Crash'.format(sev)] = Crash_DF[Crash_DF.Int_Related==0].KABCO.apply(lambda x:1 if x==sev else 0)
    Crash_DF['Ped_{}_Crash'.format(sev)] = Crash_DF['Ped{}'.format(sev)].apply(lambda x:1 if x>0 else 0)
    Crash_DF['Int_{}_Crash'.format(sev)] = Crash_DF[Crash_DF.Int_Related==1].KABCO.apply(lambda x:1 if x==sev else 0)
Crash_DF.Int_Related = Crash_DF.Int_Related.fillna(0)
    
IntR_Crash = Crash_DF[Crash_DF.Int_Related==1]
SegR_Crash = Crash_DF[Crash_DF.Int_Related==0]
PedR_Crash = Crash_DF[Crash_DF.PedInv]
for sev in ['K','A','B']:
    Res_DF['Seg{}'.format(sev)] = Res_DF.All_CIDs.apply(lambda l:Crash_DF.loc[l,'Seg_{}_Crash'.format(sev)].sum())
    Res_DF['Ped{}'.format(sev)] = Res_DF.All_CIDs.apply(lambda l:Crash_DF.loc[l,'Ped_{}_Crash'.format(sev)].sum())
    Res_DF['Int{}'.format(sev)] = Res_DF.All_CIDs.apply(lambda l:Crash_DF.loc[l,'Int_{}_Crash'.format(sev)].sum())

Res_DF['Site_ID'] = range(Res_DF.shape[0])
F = ['Site_ID','PSI_MPs', 'PSI_Mileage', 'Ped_MPs', 'Ped_Mileage', 'Num_Int', 'Int_MPs','Total_Mileage', 'Total_K','Total_A','Total_B','SegK', 'PedK', 'IntK', 'SegA', 'PedA', 'IntA', 'SegB', 'PedB', 'IntB']
for sev in ['K','A','B']:
    Res_DF['Total_{}'.format(sev)] = Res_DF['Int{}'.format(sev)] + Res_DF['Seg{}'.format(sev)]
Res_DF = Res_DF.reset_index().sort_values(by=['District','StateLocal','UrbanRural','Total_K','Total_A','Total_B'],ascending=[True,False,True,False,False,False])
Res_DF.index = pd.MultiIndex.from_arrays([Res_DF.District,Res_DF.StateLocal,Res_DF.UrbanRural,Res_DF.Dis_Label])
Res_DF[F].to_excel('Aggregated_Sites2.xlsx')
Res_DF

District  \
District   StateLocal UrbanRural        Dis_Label                                                 
District 1 State      El Paso (Partial) NM273P (MCNUTT RD)                           District 1   
                                        NM136P (PETE V DOMENICI INTERNATIONAL BLVD)  District 1   
                      Las Cruces        NM478P (S MAIN ST)                           District 1   
                                        NM188P (S VALLEY DR)                         District 1   
                                        US70P (N MAIN ST)                            District 1   
                                        US70P (W PICACHO AVE)                        District 1   
                                        NM185P (N VALLEY DR)                         District 1   
                                        NM188P (N VALLEY DR)                         District 1   
                                        NM28P (AVENIDA DE MESILLA)                   District 1   
                      Rural             US70P (US HIGHWAY 70)                        District 1   
                                        NM136P (PETE V DOMENICI INTERNATIONAL BLVD)  District 1   
                                        NM478P (S MAIN ST)                           District 1   
                                        NM185P (N VALLEY DR)                         District 1   
                      Small Urban       NM90P (N HUDSON ST)                          District 1   
                                        US180P (E US HIGHWAY 180)                    District 1   
                                        BL22P (W PINE ST;E PINE ST)                  District 1   
           Other      Las Cruces        FL5614P (E LOHMAN AVE)                       District 1   
                                        FL4505P (S SOLANO DR)                        District 1   
                                        FL5508P (UNIVERSITY)                         District 1   
                                        FL4532P (S TELSHOR BLVD)                     District 1   
                                        FL1219P (FOOTHILLS RD)                       District 1   
                                        FL5614P (W LOHMAN AVE)                       District 1   
                                        FL4539P (S TRIVIZ DR)                        District 1   
                                        FL5540P (W BOUTZ RD)                         District 1   
                                        FL5549P (SPRUCE AVE)                         District 1   
                                        FL4539P (N TRIVIZ DR)                        District 1   
                                        FR7015P (BATAAN MEMORIAL WEST)               District 1   
                                        FL5537P (S TRIVIZ DR)                        District 1   
                                        LOC251880P (NACHO DR)                        District 1   
                                        FL5558P (N ROADRUNNER PKWY)                  District 1   
...                                                                                         ...   
District 5 Other      Santa Fe          FL5804P (JAGUAR DR)                          District 5   
                                        LOC198772P (ZEPOL RD)                        District 5   
                                        FL5755P (HICKOX ST)                          District 5   
                                        LOC201262P (JUANITA ST)                      District 5   
                                        FL1429P (SILER RD)                           District 5   
                                        FL4735P (AGUA FRIA ST)                       District 5   
                      Small Urban       LOC116067P (SILKEY WAY)                      District 5   
                                        NM584P (FAIRVIEW LN)                         District 5   
                            

In [113]:
Res_V3 = pd.read_excel(Project_DIR+'\\10.Extension\\Aggregated_Sites_V3.xlsx',index_col=[0,1,2,3])
Res_DF['Selection'] = Res_V3['Selection For Further Review'].loc[Res_DF.index]


In [114]:
# Add aggregated results to GDB
df = Res_DF[['Site_ID','Shape','Selection','PSI_Mileage','Ped_Mileage','Num_Int','Total_Mileage'] + ['{}{}'.format(t,s) for t in ['Total_','Seg','Int','Ped'] for s in ['K','A','B']]]
df.Ped_Mileage = df.Ped_Mileage.fillna(0).round(2)
for c in ['{}{}'.format(t,s) for t in ['Seg','Int','Ped'] for s in ['K','A','B']]:
    df[c] = df[c].fillna(0).astype(int)
df = df.reset_index()
hsmpy3.common.ConvertPandasDFtoTable(df,ResultsGDB+'\\Agg_Locations',shape='Shape',spatialReference=hsmpy3.nm.NAD83_NM)

R_Agg_Crash = ResultsGDB + '\\R_Agg_Crash'
arcpy.Delete_management(R_Agg_Crash)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\Agg_Locations',
    destination_table      = ResultsGDB + '\\NM_Crash_KAB_12_16', 
    out_relationship_class = R_Agg_Crash, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Crash", 
    backward_label         = "Agg_Locations", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "Site_ID", 
    origin_foreign_key     = "Site_ID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)

df = pd.DataFrame([(r.Site_ID,cid) for i,r in Res_DF.iterrows() for cid in r.All_CIDs],columns=['Site_ID','CID'])
df.index = df.Site_ID
Agg_Crash_DF = df
uc = arcpy.InsertCursor(R_Agg_Crash)
for i,r in Agg_Crash_DF.iterrows():
    row = uc.newRow()
    row.setValue('Site_ID',r.Site_ID)
    row.setValue('CID',r.CID)
    uc.insertRow(row)
del uc
print('[{}] Agg - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

[2019-04-17 12:18:30] create table
[2019-04-17 12:18:30] deleting current table: 447
[2019-04-17 12:18:38] spatial reference passed: NAD_1983_2011_StatePlane_New_Mexico_Central_FIPS_3002_Ft_US
[2019-04-17 12:18:38] geometry detected: POLYLINE
[2019-04-17 12:18:41] adding fields
[2019-04-17 12:19:53] inserting rows
[2019-04-17 12:19:55] done! 447
[2019-04-17 12:20:08] Agg - Crash done!


In [11]:
# Create Figures for each Aggreagted Site
print(strftime("%Y-%m-%d %H:%M:%S"))
def SHSP_EABarStateLine(Crash_DF,Selection,Field,png_out,Order=[],Length=0,r=''):
    df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
    df.KAB = df.KAB/float(sum(df.KAB))
    df.columns = [Field]
    if len(Order) == 0:
        df = df.sort_index(ascending=True)
    else:
        df = df.loc[Order]
    if 1<2:
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        eadf = pd.DataFrame(Crash_DF.loc[Selection].groupby([Field])['KAB'].aggregate(sum))
        eadf = eadf.fillna(0)
        eadf.KAB = eadf.KAB/float(sum(eadf.KAB))
        eadf.columns = [Field]
        if Length>0:
            eadf = eadf.sort_values(Field,ascending=False)
            eadf = eadf.iloc[range(min(Length,len(eadf)))]
            df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
            df.KAB = df.KAB/float(sum(df.KAB))
            df.columns = [Field]
            if len(Order) == 0:
                df = df.sort_index(ascending=True)
            else:
                df = df.loc[Order]
            df  = df.loc[list(eadf.index)]
        else:
            eadf = eadf.loc[df.index]
        eadf = eadf.fillna(0)
        V = [v1-v2 for v1,v2 in zip(eadf[Field],df[Field])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(eadf)),eadf[Field],align='center',color=my_cmap(my_norm(V)))
        plt.xticks(range(len(df)),df.index,rotation=90)
        p2, = plt.plot(range(len(df)),df[Field],'-o',color='green')
        plt.xlabel(Field)
        #plt.xticks(rotation=90)
        plt.title('{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
        plt.grid()
        plt.gca().set_yticklabels(['{:,.2%}'.format(x) for x in plt.gca().get_yticks()])
        plt.legend([p2,p1],['Statewide Crashes ({})'.format(Crash_DF.shape[0]),'Site-Specific Crashes ({})'.format(len(Selection))],loc=2,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.tight_layout()
        plt.savefig(png_out,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close()
def SHSP_TimeTrend_Contour(DF,PNGName,Title):
    import warnings
    warnings.filterwarnings('ignore')
    import matplotlib.pyplot as plt
    DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in DF.Date]
    TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
    TimeOrder.reverse()
    DF['DayName'] = [d.weekday_name for d in DF.Date]
    DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    DF['Month']  = [d.strftime('%b') for d in DF.Date]
    MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    if 1<2:
        df0 = pd.DataFrame(DF.groupby(['Month','DayName','Time'])['KAB'].aggregate(sum))
        df0 = df0.unstack('Time').T
        df0.index = df0.index.droplevel(0)
        arr = []
        for m in MonthOrder:
            for d in DayOrder:
                arr.append((m,d))
        cs = pd.MultiIndex.from_tuples(arr)
        df = pd.DataFrame(columns=cs,index=TimeOrder)
        for i1 in df.index:
            for j1 in df.columns:
                try:
                    df.set_value(i1,j1,int(df0.loc[i1,j1]))
                except:
                    pass
        df = df.fillna(0)
        df = df.loc[TimeOrder]
        L = []
        for l in list(df.values):
            L.extend(l)
        v_max = max(L)
        v_min = min(L)
        df = df/v_max
        v_max = 1
        v_min = 0
    
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        ax1 = plt.subplot(211)
    
        df1 = df[MonthOrder[0:6]]
        #pl1 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl1 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        xposition = [7*(i1)-0.5 for i1 in range(1,6)]
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[1][0:2] + '  ' + f[0],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.gca().xaxis.tick_top()
        plt.yticks(range(1,len(df1.index)),list(df1.index)[1:],rotation=0,fontsize=6)
        
        plt.grid()
        df1 = df[MonthOrder[6:13]]
        ax2 = plt.subplot(212)
        #pl2 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl2 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[0] + '  ' + f[1][0:2],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.yticks(range(len(df1.index)),df1.index,rotation=0,fontsize=6)
        plt.grid()
        #plt.gcf().subplots_adjust(right=0.8)
        #cbar_ax = plt.gcf().add_axes([0.85, 0.15, 0.05, 0.7])
        #plt.colorbar(pl1, cax=cbar_ax)
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.suptitle(Title,y=0.99)
        #plt.tight_layout()
        plt.savefig(PNGName,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close('all')
        del ax1
        del ax2
        del pl1
        del pl2

f = IntProgress(min=0,max=Res_DF.shape[0],value=0)
display(f)
for i,r in Res_DF.iterrows():
    SHSP_TimeTrend_Contour(Crash_DF.loc[r.All_CIDs],'Site_{}_Contour.png'.format(r.Site_ID),'{} - {} - {} - {}\n({} KAB Crashes)'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label,Crash_DF.loc[r.All_CIDs].shape[0]))
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Time','Site_{}_Time.png'.format(r.Site_ID),Order=TimeOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'DayName','Site_{}_Day.png'.format(r.Site_ID),Order=DayOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Month','Site_{}_Month.png'.format(r.Site_ID),Order=MonthOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'DayNight','Site_{}_DayNight.png'.format(r.Site_ID),Order=['Dawn','Day','Dusk','Night'],Length=0,r=r)
    #SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Light','Light.png',Order=[],Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Weather','Site_{}_Weather.png'.format(r.Site_ID),Order=[],Length=0,r=r)
    f.value += 1
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-26 10:18:39


IntProgress(value=0, max=447)

MemoryError: 

MemoryError: 

<Figure size 5200x2800 with 2 Axes>

In [168]:
# Add Agg Locations Attachments
print('[{}] read hsip attachments'.format(strftime("%Y-%m-%d %H:%M:%S")))
df = pd.DataFrame(index = pd.MultiIndex.from_product([Res_DF.Site_ID,
                                                      ['Time','Day','Month','Contour','Weather','EASeverity','TypeSeverity']],
                                                     names=['Site_ID','Figure']),
                  data={'Path':PythonDir}).reset_index()
df.index = df.Site_ID
arcpy.Delete_management(ResultsGDB + '\\Agg_Locations_Att')
arcpy.CreateTable_management(out_path=ResultsGDB,out_name='Agg_Locations_Att')
arcpy.AddField_management(in_table=ResultsGDB + '\\Agg_Locations_Att',field_name='Site_ID',field_type='Long',field_length=15)
arcpy.AddField_management(in_table=ResultsGDB + '\\Agg_Locations_Att',field_name='Path',field_type='Text',field_length=500)
uc = arcpy.InsertCursor(ResultsGDB + '\\Agg_Locations_Att')
for i,r in df.iterrows():
    row = uc.newRow()
    row.setValue('Site_ID',r.Site_ID)
    row.setValue('Path',os.path.join(r.Path,'Site_{}_{}.png'.format(r.Site_ID,r.Figure)))
    uc.insertRow(row)
del uc
HSIP_Table = 'Agg_Locations'
print('[{}]  - {}'.format(strftime("%Y-%m-%d %H:%M:%S"),df.shape[0]))
print('[{}] enabling attachments'.format(strftime("%Y-%m-%d %H:%M:%S")))
arcpy.EnableAttachments_management (os.path.join(ResultsGDB ,HSIP_Table))
print('[{}] adding attachments'.format(strftime("%Y-%m-%d %H:%M:%S")))
arcpy.AddAttachments_management(
    in_dataset=os.path.join(ResultsGDB ,HSIP_Table), 
    in_join_field='Site_ID', 
    in_match_table=ResultsGDB + '\\Agg_Locations_Att', 
    in_match_join_field='Site_ID', 
    in_match_path_field='Path'
)
arcpy.Delete_management(ResultsGDB + '\\Agg_Locations_Att')
print('[{}] done'.format(strftime("%Y-%m-%d %H:%M:%S")))

[2019-04-17 14:15:08] read hsip attachments
[2019-04-17 14:15:24]  - 3129
[2019-04-17 14:15:24] enabling attachments
[2019-04-17 14:15:27] adding attachments
[2019-04-17 14:17:47] done


In [14]:
path_to_domains = r'\\chcfpp01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\10.Extension\NM_Domains.xlsx'
df = pd.read_excel(path_to_domains)
def TranslateDomains_CSV(DF,DomainCSV,subset=[]):
    df = pd.DataFrame(columns=['field_name'])
    if os.path.basename(DomainCSV).split('.')[-1].lower()=='csv':
        df = pd.read_csv(DomainCSV)
    if os.path.basename(DomainCSV).split('.')[-1].lower() in ['xls','xlsx']:
        df = pd.read_excel(DomainCSV)
    if len(subset)==0:
        subset = DF.columns
    else:
        subset = [c for c in subset if c in DF.columns]
    for c in subset:
        if c in df.field_name.unique():
            idf = df[df.field_name=='Class'][['code','description']]
            idf.index = idf.code
            d = idf.description.to_dict()
            if len(d)>0:
                DF[c] = DF[c].apply(lambda x:'{}. {}'.format(x,d[x]) if x in d.keys() else x)
    return(DF)
Crash_DF['Class_T'] = TranslateDomains_CSV(Crash_DF[['Class']].fillna(99).astype(int),path_to_domains)
df = pd.DataFrame(Crash_DF.groupby(['Class_T','KABCO']).size()).unstack().fillna(0).astype(int)
df.columns = df.columns.droplevel(0)
df = df.sort_values(['K','A','B'],ascending=False)
df = df[['K','A','B']]
df

KABCO,K,A,B
Class_T,,,
4. Other Vehicle,476,2835,8099
1. Overturn,372,665,2138
3. Pedestrian,311,451,854
12. Rollover,160,226,528
10. Fixed Object,156,579,2080
99. Left Blank,82,106,257
8. Pedalcyclist,25,136,676
2. Other non-collision,23,129,494
5. Vehicle on the roadway,16,31,99


In [15]:
F = ['PedInv','PECinv','Druginv','Alcinv','MCinv','TRKinv','HZinv','OnCurve','TribalJurisdiction','Int_Related']
Des =  ['Pedestrian Involved', 'Pedalcyclist Involved','Drug Involved','Alcohol Involved','Motorcylce Involved','Truck Involved',
'Hazardous Material Involved','On the Curve','Tribal Jurisdiction','Intersection Related']
def EASeverity(All_Crashes_DF,SelectionIndex,EAFields = [],Descriptions = [],SeverityField='KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    for f in EAFields:
        All_Crashes_DF[f] = All_Crashes_DF[f].apply(lambda x:1 if x==1 else 0)

    All_DF = Crash_DF.groupby(SeverityField)[EAFields].sum().T
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()
    
    
    Sel_DF = Crash_DF.loc[SelectionIndex].groupby(SeverityField)[EAFields].sum().T
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    All_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    Sel_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,Crash_DF[Crash_DF[SeverityField]==label].shape[0]),
                            'Selected {} Crashes ({})'.format(label,Crash_DF.loc[SelectionIndex][Crash_DF[SeverityField]==label].shape[0])],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()

def CrashTypeSeverity(All_Crashes_DF,SelectionIndex,TypeField = 'Class_T',SeverityField = 'KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    All_DF = pd.DataFrame(All_Crashes_DF.groupby([TypeField,SeverityField]).size()).unstack().fillna(0).astype(int)
    All_DF.columns = All_DF.columns.droplevel(0)
    All_DF = All_DF[Order]
    All_DF = All_DF.sort_values(Order,ascending=False)
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()

    Sel_DF = pd.DataFrame(All_Crashes_DF.loc[SelectionIndex].groupby([TypeField,SeverityField]).size()).unstack()
    Sel_DF.columns = Sel_DF.columns.droplevel(0)
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,All_DF[label].sum()),
                            'Selected {} Crashes ({})'.format(label,Sel_DF[label].sum())],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()
    plt.clf()
print(strftime("%Y-%m-%d %H:%M:%S"))
f = IntProgress(min=0,max=Res_DF.shape[0],value=0)
display(f)
for i,r in Res_DF.iterrows():
    CrashTypeSeverity(Crash_DF,Res_DF.All_CIDs.loc[i],'Class_T','KABCO',['K','A','B'], PNGOut='Site_{}_TypeSeverity.png'.format(r.Site_ID),
                  title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
    EASeverity(Crash_DF,Res_DF.All_CIDs.loc[i],F,Des,SeverityField='KABCO',Order = ['K','A','B'], PNGOut='Site_{}_EASeverity.png'.format(r.Site_ID),
               title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
    f.value+=1
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-25 11:54:08


IntProgress(value=0, max=447)

2019-04-25 13:24:04
